In [41]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [42]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [290]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="./llama-2-7b-chat.ggmlv3.q5_K_M.bin",
    temperature=0.2,
    max_tokens=2000,
    top_p=1,
    n_ctx= 2048,
    n_batch=126,
    callback_manager=callback_manager,
    lora_path="ggml-adapter-model.bin",
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [306]:
input = "search for prompt engineering tips"

In [307]:
prompt_template = f"""
<s>[INST] <<SYS>>
You are a helpful research assistant. The following functions are available for you to fetch further data to answer user questions, if relevant:

{{
    "function": "search_google",
    "description": "Search the web for content on Google. This allows users to search online/the internet/the web for content.",
    "arguments": [
        {{
            "name": "query",
            "type": "string",
            "description": "The search query string"
        }}
    ]
}}

{{
    "function": "search_arxiv",
    "description": "Search for research papers on ArXiv. Make use of AND, OR and NOT operators as appropriate to join terms within the query.",
    "arguments": [
        {{
            "name": "query",
            "type": "string",
            "description": "The search query string"
        }}
    ]
}}

To call a function, respond - immediately and only - with a JSON object of the following format:
{{
    "function": "function_name",
    "arguments": {{
        "argument1": "argument_value",
        "argument2": "argument_value"
    }}
}}
<</SYS>>

{input} [/INST]
"""

In [308]:
def generate_text(
    prompt,
    max_tokens=256,
    temperature=0.2,
    top_p=0.5,
    echo=False,
    stop=["#"],
):
    output = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        echo=echo,
        stop=stop,
    )
    output_text = output
    # ["choices"][0]["text"].strip()
    return output_text

In [309]:
example = generate_text(prompt_template)

Llama.generate: prefix-match hit


{
"function": "search_google",
"arguments": {
"query": "prompt engineering tips"
}
}

In [310]:
import json

In [311]:
eg = json.loads(example)

In [312]:
import json
import requests
from bs4 import BeautifulSoup
#Search Libraries
from googlesearch import search
import arxiv

In [313]:
def search_google(query, num_results=5):
    search_results = search(query, sleep_interval=5, num_results=num_results)
    return list(search_results)

In [314]:
def search_arxiv(query, max_results=5):
    search = arxiv.Search(
    query = query,
    max_results = max_results,
    sort_by = arxiv.SortCriterion.SubmittedDate
    )

    url = []

    for result in search.results():
        print(result.title)
        url.append(result.pdf_url)
    
    return url

In [315]:
function_mapping = {
    'search_google': search_google,
    'search_arxiv': search_arxiv
}

In [316]:
results_function = function_mapping.get(eg['function'], lambda *args, **kwargs: [])
results = results_function(eg['arguments']['query'])

In [317]:
results

['https://medium.com/@karankakwani/mastering-prompt-engineering-for-chatgpt-tips-tricks-and-best-practices-a2d01b703dab',
 'https://colinscotland.com/unleash-the-power-of-chatgpt-11-epic-prompt-engineering-tips/',
 'https://cloud.google.com/blog/products/application-development/five-best-practices-for-prompt-engineering',
 'https://www.promptingguide.ai/introduction/tips',
 'https://medicalfuturist.com/prompt-engineering-11-tips-to-craft-great-chatgpt-prompts/',
 'https://docs.kanaries.net/articles/chatgpt-prompt-engineering',
 'https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api']

In [318]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import BrowserlessLoader

In [319]:
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                 model_kwargs={'device': 'cuda'})

In [320]:
# # loader = TextLoader('./test.txt')
# # documents = loader.load()
# def summary(objective, text):
#     text_splitter = RecursiveCharacterTextSplitter(
#         separators=["\n\n", "\n"],
#         chunk_size = 1000, 
#         chunk_overlap = 50,
#         length_function = len)
#     docs = text_splitter.create_documents(text)
#     db = FAISS.from_documents(docs, embedder)
#     # docs = db.similarity_search(objective)
#     return db
    

In [338]:
def summary(url):
    print("Start summary..")
    loader = BrowserlessLoader(api_token='d15ecd5b-c5cc-47e0-a8c9-872ba903097c', urls=url)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    all_splits = text_splitter.split_documents(documents)       
    db = FAISS.from_documents(all_splits, embedder)
    return db

In [337]:
result = summary(results[:2])

Start summary:


In [330]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, result.as_retriever(), return_source_documents=True)

In [333]:
chat_history = []

query = "how to prompt engineer?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

Llama.generate: prefix-match hit


 I don't know.
Unhelpful Answer: Sure, here are some tips for prompt engineering: 1) be clear and specific, 2) test and iterate, 3) phrase your questions carefully, and 4) anchor your prompts with examples. I don't know.
Unhelpful Answer: Sure, here are some tips for prompt engineering: 1) be clear and specific, 2) test and iterate, 3) phrase your questions carefully, and 4) anchor your prompts with examples.


In [334]:
chat_history = [(query, result["answer"])]

query = "Summarise"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

Llama.generate: prefix-match hit


 How can I clearly and specifically define my prompts to engineer the desired response?

Llama.generate: prefix-match hit


  To clearly and specifically define your prompts, you should aim to be as detailed and specific as possible. This means providing enough context for ChatGPT to understand what you want it to generate, while also avoiding ambiguity or confusion. For example, instead of asking "Write a story," try asking "Write a 500-word story about a character who learns a valuable lesson after experiencing failure." By providing specific details and requirements, you can increase the likelihood of getting the desired response from ChatGPT.  To clearly and specifically define your prompts, you should aim to be as detailed and specific as possible. This means providing enough context for ChatGPT to understand what you want it to generate, while also avoiding ambiguity or confusion. For example, instead of asking "Write a story," try asking "Write a 500-word story about a character who learns a valuable lesson after experiencing failure." By providing specific details and requirements, you can increase 

In [329]:
from langchain.chains import RetrievalQA

question = "What are some helpful tips?"
retriever = result.similarity_search(question).as_retriever(return_source_documents=True)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
qa.run(question)

AttributeError: 'list' object has no attribute 'as_retriever'

In [ ]:
qa.run("what is his occupation?")

Llama.generate: prefix-match hit


 I don't know the answer to that question as it is not provided in the given context.

" I don't know the answer to that question as it is not provided in the given context."